# Projeto 1 - Ciência dos Dados

Nome: Gabriel Tkacz

Nome: Guilherme Carvalho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [185]:
!pip install emoji
import emoji
import functools
import operator
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
pd.options.display.max_rows = 13

In [186]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Teka\Documents\Insper\CDados\projeto1cdados-master (1)\projeto1cdados-master


In [187]:
def cleanup(text):
    punctuation='[!-.:?;#…\/]' # Note que os sinais [] são delimitadores de um conjunto.
    lixo=['(', ')', '[',']', '\n', '\t','rt']
    pattern=re.compile(punctuation)
    text_subbed1=re.sub(r"http\S+", "", text)
    text_subbed=re.sub(pattern, ' ', text_subbed1)
    text_subbed=re.sub(r'()@\w+', r'', text_subbed)
    text_subbed=re.sub(r'@\w+', r'', text_subbed)
    for i in lixo:
        text_subbed=text_subbed.replace(i,'')
    text_subbed=text_subbed.replace('á','a')
    text_subbed=text_subbed.replace('ã','a')
    text_subbed=text_subbed.replace('â','a')
    text_subbed=text_subbed.replace('é','e')
    text_subbed=text_subbed.replace('ê','e')
    text_subbed=text_subbed.replace('í','i')
    text_subbed=text_subbed.replace('ô','o')
    text_subbed=text_subbed.replace('ó','o')
    text_subbed=text_subbed.replace('ú','u')
    text_subbed=text_subbed.replace('ç','c')
    text_subbed=text_subbed.replace('ñ','nao')
    text_subbed=text_subbed.replace('vc','voce')
    '''text_subbed=emoji.get_emoji_regexp()(text_subbed)
    text_subbed=[substr.split() for substr in text_subbed]
    text_subbed=functools.reduce(operator.concat, text_subbed)'''
    
    return text_subbed

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [188]:
dados = pd.read_excel('NuBank.xlsx')
dados

,Treinamento,Classificação
0,@jaum_lealf @cecbreno @nubank amigo que triste...,0
1,"@oandrevinic aaaah, que notícia boa!\n\nconta ...",0
2,quando o nubank vai liberar a minha função cré...,0
3,@catundabruno @nubank eu vou cancelar. não hou...,0
4,"nubank aumentou meu limite, to fudida 🥺😭😂",0
...,...,...
295,se tem uma coisa que eu amo é meu cartão nuban...,0
296,rt @wjucollection: 🌱 itzy\n\nalbum it’z icy: f...,0
297,aff meu nubank novo é mt lindo,0
298,"olhei o dólar canadense hoje e tá r$ 3,39 e ta...",0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [189]:
#definindo variavel categorica
dados.loc[:,'Classificação'] = dados['Classificação'].astype('category')
dados['Classificação'].cat.categories = ["Irrelevante", "Relevante"]
dados

,Treinamento,Classificação
0,@jaum_lealf @cecbreno @nubank amigo que triste...,Irrelevante
1,"@oandrevinic aaaah, que notícia boa!\n\nconta ...",Irrelevante
2,quando o nubank vai liberar a minha função cré...,Irrelevante
3,@catundabruno @nubank eu vou cancelar. não hou...,Irrelevante
4,"nubank aumentou meu limite, to fudida 🥺😭😂",Irrelevante
...,...,...
295,se tem uma coisa que eu amo é meu cartão nuban...,Irrelevante
296,rt @wjucollection: 🌱 itzy\n\nalbum it’z icy: f...,Irrelevante
297,aff meu nubank novo é mt lindo,Irrelevante
298,"olhei o dólar canadense hoje e tá r$ 3,39 e ta...",Irrelevante


In [190]:
#tweets relevantes:
rel = dados["Classificação"] == "Relevante"
relevantes = dados.loc[rel,["Treinamento"]].copy()
relevantes

,Treinamento
7,"@rodrigo_rhr @nubank @bradesco aff, pistola es..."
8,"app da digo é uma bosta, mil vezes nubank"
11,@newspotgabs @nubank so queria que me dessem l...
12,@vitorswu @nubank @bradesco nossa direto os ca...
14,chegou o meu cartão da nubank aaaaa 😍😍
...,...
251,"@nubank podia aumentar meu limite, não está da..."
258,"@minimesq sim, e ai?? vai aparecer quando?\nre..."
265,nubank ja pode libera meu cartão
289,@nubank poderia rolar um aumento de comemoraçã...


In [191]:
#tweets irrelevantes:
irel = dados["Classificação"] == "Irrelevante"
irrelevantes = dados.loc[irel,["Treinamento"]].copy()
irrelevantes

,Treinamento
0,@jaum_lealf @cecbreno @nubank amigo que triste...
1,"@oandrevinic aaaah, que notícia boa!\n\nconta ..."
2,quando o nubank vai liberar a minha função cré...
3,@catundabruno @nubank eu vou cancelar. não hou...
4,"nubank aumentou meu limite, to fudida 🥺😭😂"
...,...
294,@luscamantovani @nubank a pisciana kkkk
295,se tem uma coisa que eu amo é meu cartão nuban...
296,rt @wjucollection: 🌱 itzy\n\nalbum it’z icy: f...
297,aff meu nubank novo é mt lindo


In [192]:
lista_relevante= relevantes["Treinamento"].tolist()
lista_relevante

['@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.',
 'app da digo é uma bosta, mil vezes nubank',
 '@newspotgabs @nubank so queria que me dessem limite',
 '@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalho',
 'chegou o meu cartão da nubank aaaaa 😍😍',
 'até hj o novo cartão não chegou. quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais.',
 '@malualcure1 @nubank taxa 6,50 no saque pra quebrar a cara e por a frase em efeito (não existe nada perfeito nesse mundo)',
 'eu que lute pra krl pra pagar essa porra, mdssssss quero meu nubank de volta https://t.co/viz1jeeeea',
 'momento raro pra mim: vou elogiar algo.\nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual, bloquear a transação e o cartão e me avisar imediatamente. :)\nñ é fácil definir meu padrão 

In [193]:
lista_irrelevante= irrelevantes["Treinamento"].tolist()

In [194]:
#criandotxt
lista_relevante2 = ''.join(lista_relevante)
lista_relevante2

'@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente da @nubank disse q não tinha nada disso registrado e q eu tinha cancelado o cartão pq não queria mais.@malualcure1 @nubank taxa 6,50 no saque pra quebrar a cara e por a frase em efeito (não existe nada perfeito nesse mundo)eu que lute pra krl pra pagar essa porra, mdssssss quero meu nubank de volta https://t.co/viz1jeeeeamomento raro pra mim: vou elogiar algo.\nprbns pro sistema do @nubank que conseguiu detectar uma fraude no meu cartão de crédito virtual, bloquear a transação e o cartão e me avisar imediatamente. :)\nñ é fácil definir meu padrão de compras, ainda mais com 3 pessoas util

In [195]:
lista_irrelevante2 = ''.join(lista_irrelevante)
lista_irrelevante2

'@jaum_lealf @cecbreno @nubank amigo que triste, força que eles vão aumentar!! segue os conselhos deles@oandrevinic aaaah, que notícia boa!\n\nconta para a gente, como você vai comemorar? 💜 https://t.co/jh4xqbfn94quando o nubank vai liberar a minha função crédito? nem eles sabem 🤷🏻\u200d♂️@catundabruno @nubank eu vou cancelar. não houve prejuízo financeiro, mas se houvesse, como ia ser com esse atendimento?nubank aumentou meu limite, to fudida 🥺😭😂aí @nubank pq meu cartão não vai no @uber_brasil?rt @capyvaro: se és tão bonzinho nubank, pq cobraste minha fatura desse mês.@joaoloque não deixe o cartão do nubank com essa mulher@algudaodoce69 @nubank @nvidiageforcebr será que vai dar pra você comer um méqui enquanto testa a placa de vídeo? 🤔😂@tiagoocaetano @nubank já pensou. complicado demais.@cecbreno @nubank e eu só quero enxergar bem sem ter que usar óculos kkkkkkk@nubank , posso pagar a fatura do cartão com o próprio cartão ? #stonks 💷@nubank aumenta meu limite agoraaalguém tem conta nu

In [196]:
#ceiando arquivo texto relevantes:
arquivo = open("arqR.txt","w",encoding="utf-8-sig")
arquivo.write(lista_relevante2)
arquivo.close()

In [197]:
#criando arquivo texto irrelevantes:
arquivo = open("arqIR.txt","w",encoding="utf-8-sig")
arquivo.write(lista_irrelevante2)
arquivo.close()

In [198]:
#lendo arquivo texto relevantes:
with open("arqR.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    texto_relevante = arquivo_texto.read()

# Imprime os 100 primeiros caracteres do texto para verificar se está tudo ok.
print(texto_relevante[0:404])

@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente


In [199]:
#lendo arquivo texto irrelevantes:
with open("arqIR.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    texto_irrelevante = arquivo_texto.read()

# Imprime os 100 primeiros caracteres do texto para verificar se está tudo ok.
print(texto_relevante[0:404])

@rodrigo_rhr @nubank @bradesco aff, pistola estou kkk se eu estivesse devendo ou com score baixo até entenderia.app da digo é uma bosta, mil vezes nubank@newspotgabs @nubank so queria que me dessem limite@vitorswu @nubank @bradesco nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu cartão da nubank aaaaa 😍😍até hj o novo cartão não chegou. quando fui perguntar a atendente


In [200]:
texto_R_clean = cleanup(texto_relevante)
texto_R_clean

'   aff  pistola estou kkk se eu estivesse devendo ou com score baixo ate entenderia app da digo e uma bosta  mil vezes nubank  so queria que me dessem limite   nossa direto os caras aumentam meu limite kk e olha q eu nem trabalhochegou o meu caao da nubank aaaaa 😍😍ate hj o novo caao nao chegou  quando fui perguntar a atendente da  disse q nao tinha nada disso registrado e q eu tinha cancelado o caao pq nao queria mais   taxa 6 50 no saque pra quebrar a cara e por a frase em efeito  nao existe nada perfeito nesse mundo eu que lute pra krl pra pagar essa porra  mdssssss quero meu nubank de volta  raro pra mim  vou elogiar algo prbns pro sistema do  que conseguiu detectar uma fraude no meu caao de credito viual  bloquear a transacao e o caao e me avisar imediatamente    nao e facil definir meu padrao de compras  ainda mais com 3 pessoas utilizando   mais um mes pagando o nubank adiantado e ele me negando aumento de limite 👀 libere meu limite  eu tenho pessoas pra pagar  ate que eu faria 

In [201]:
texto_IR_clean = cleanup(texto_irrelevante)
texto_IR_clean

'   amigo que triste  forca que eles vao aumentar   segue os conselhos deles aaaah  que noticia boa conta para a gente  como voce vai comemorar  💜  o nubank vai liberar a minha funcao credito  nem eles sabem 🤷🏻\u200d♂️  eu vou cancelar  nao houve prejuizo financeiro  mas se houvesse  como ia ser com esse atendimento nubank aumentou meu limite  to fudida 🥺😭😂ai  pq meu caao nao vai no     se es tao bonzinho nubank  pq cobraste minha fatura desse mes  nao deixe o caao do nubank com essa mulher   sera que vai dar pra voce comer um mequi enquanto testa a placa de video  🤔😂  ja pensou  complicado demais   e eu so quero enxergar bem sem ter que usar oculos kkkkkkk   posso pagar a fatura do caao com o proprio caao    stonks 💷 aumenta meu limite agoraaalguem tem conta nubank pra me tirar uma duvida  se e possivel sacar sem caao🌱 4minutealbum act 7  photocard da sohyun  album raro e fora de producaor 120 com frete incluso🌱 possuo feedback positivo 🌱 envio para todo brasil 🌱 aceito picpay  itau  

In [202]:
texto_total = texto_R_clean + texto_IR_clean

In [203]:
totdas_palavras_relevantes = texto_R_clean.split()
totdas_palavras_relevantes

['aff',
 'pistola',
 'estou',
 'kkk',
 'se',
 'eu',
 'estivesse',
 'devendo',
 'ou',
 'com',
 'score',
 'baixo',
 'ate',
 'entenderia',
 'app',
 'da',
 'digo',
 'e',
 'uma',
 'bosta',
 'mil',
 'vezes',
 'nubank',
 'so',
 'queria',
 'que',
 'me',
 'dessem',
 'limite',
 'nossa',
 'direto',
 'os',
 'caras',
 'aumentam',
 'meu',
 'limite',
 'kk',
 'e',
 'olha',
 'q',
 'eu',
 'nem',
 'trabalhochegou',
 'o',
 'meu',
 'caao',
 'da',
 'nubank',
 'aaaaa',
 '😍😍ate',
 'hj',
 'o',
 'novo',
 'caao',
 'nao',
 'chegou',
 'quando',
 'fui',
 'perguntar',
 'a',
 'atendente',
 'da',
 'disse',
 'q',
 'nao',
 'tinha',
 'nada',
 'disso',
 'registrado',
 'e',
 'q',
 'eu',
 'tinha',
 'cancelado',
 'o',
 'caao',
 'pq',
 'nao',
 'queria',
 'mais',
 'taxa',
 '6',
 '50',
 'no',
 'saque',
 'pra',
 'quebrar',
 'a',
 'cara',
 'e',
 'por',
 'a',
 'frase',
 'em',
 'efeito',
 'nao',
 'existe',
 'nada',
 'perfeito',
 'nesse',
 'mundo',
 'eu',
 'que',
 'lute',
 'pra',
 'krl',
 'pra',
 'pagar',
 'essa',
 'porra',
 'mdssss

In [204]:
totdas_palavras_irrelevantes = texto_IR_clean.split()
totdas_palavras_irrelevantes

['amigo',
 'que',
 'triste',
 'forca',
 'que',
 'eles',
 'vao',
 'aumentar',
 'segue',
 'os',
 'conselhos',
 'deles',
 'aaaah',
 'que',
 'noticia',
 'boa',
 'conta',
 'para',
 'a',
 'gente',
 'como',
 'voce',
 'vai',
 'comemorar',
 '💜',
 'o',
 'nubank',
 'vai',
 'liberar',
 'a',
 'minha',
 'funcao',
 'credito',
 'nem',
 'eles',
 'sabem',
 '🤷🏻\u200d♂️',
 'eu',
 'vou',
 'cancelar',
 'nao',
 'houve',
 'prejuizo',
 'financeiro',
 'mas',
 'se',
 'houvesse',
 'como',
 'ia',
 'ser',
 'com',
 'esse',
 'atendimento',
 'nubank',
 'aumentou',
 'meu',
 'limite',
 'to',
 'fudida',
 '🥺😭😂ai',
 'pq',
 'meu',
 'caao',
 'nao',
 'vai',
 'no',
 'se',
 'es',
 'tao',
 'bonzinho',
 'nubank',
 'pq',
 'cobraste',
 'minha',
 'fatura',
 'desse',
 'mes',
 'nao',
 'deixe',
 'o',
 'caao',
 'do',
 'nubank',
 'com',
 'essa',
 'mulher',
 'sera',
 'que',
 'vai',
 'dar',
 'pra',
 'voce',
 'comer',
 'um',
 'mequi',
 'enquanto',
 'testa',
 'a',
 'placa',
 'de',
 'video',
 '🤔😂',
 'ja',
 'pensou',
 'complicado',
 'demais',


In [205]:
serie_relevantes = pd.Series(totdas_palavras_relevantes)
serie_relevantes

0          aff
1      pistola
2        estou
3          kkk
4           se
        ...   
845      custa
846         me
847    aceitar
848         no
849    credito
Length: 850, dtype: object

In [206]:
serie_irrelevantes = pd.Series(totdas_palavras_irrelevantes)
serie_irrelevantes

0          amigo
1            que
2         triste
3          forca
4            que
          ...   
3689           e
3690          me
3691    cobraram
3692        isso
3693          pq
Length: 3694, dtype: object

In [207]:
tabela_relevantes  = serie_relevantes.value_counts()
tabela_relevantes

e             35
o             28
meu           26
de            26
que           20
              ..
pqp            1
rendendo       1
kkkkkkkkkk     1
utilidade      1
digo           1
Length: 395, dtype: int64

In [208]:
tabela_relevantes_relativa = serie_relevantes.value_counts(True)
tabela_relevantes_relativa

e             0.041176
o             0.032941
meu           0.030588
de            0.030588
que           0.023529
                ...   
pqp           0.001176
rendendo      0.001176
kkkkkkkkkk    0.001176
utilidade     0.001176
digo          0.001176
Length: 395, dtype: float64

In [209]:
tabela_irrelevantes_relativa = serie_irrelevantes.value_counts(True)
tabela_irrelevantes_relativa

e               0.030590
que             0.027342
o               0.026530
de              0.025176
nubank          0.024635
                  ...   
mesmoajustei    0.000271
seriam          0.000271
deija           0.000271
liberam         0.000271
digo            0.000271
Length: 1255, dtype: float64

In [210]:
serie_Total = pd.Series(texto_total.split())
tabela_Total_relativa = serie_Total.value_counts(True)
tabela_Total_relativa

e         0.032570
o         0.027729
que       0.026629
de        0.026188
nubank    0.023548
            ...   
viagem    0.000220
entro     0.000220
kkkkq     0.000220
tiver     0.000220
bala      0.000220
Length: 1427, dtype: float64

In [211]:
#verificando a que texto o tweet se refere:
def relevancia(frase):
    frase = cleanup(frase.lower())
    P_R_dado_TT = tabela_relevantes_relativa[frase.split()].prod()
    P_IR_dadoTT = tabela_irrelevantes_relativa[frase.split()].prod()
    if P_R_dado_TT>P_IR_dadoTT:
        return 'Relevante'
    else:
        return 'Irrelevante'

In [212]:
dados

,Treinamento,Classificação
0,@jaum_lealf @cecbreno @nubank amigo que triste...,Irrelevante
1,"@oandrevinic aaaah, que notícia boa!\n\nconta ...",Irrelevante
2,quando o nubank vai liberar a minha função cré...,Irrelevante
3,@catundabruno @nubank eu vou cancelar. não hou...,Irrelevante
4,"nubank aumentou meu limite, to fudida 🥺😭😂",Irrelevante
...,...,...
295,se tem uma coisa que eu amo é meu cartão nuban...,Irrelevante
296,rt @wjucollection: 🌱 itzy\n\nalbum it’z icy: f...,Irrelevante
297,aff meu nubank novo é mt lindo,Irrelevante
298,"olhei o dólar canadense hoje e tá r$ 3,39 e ta...",Irrelevante


In [213]:
#Teste:
for i in range(len(dados.Treinamento)):
    tt =(dados.Treinamento[i])
    tweet = tt.split()
    for pal in tweet:
        if pal not in totdas_palavras_relevantes:
            totdas_palavras_relevantes.append(pal)
        if pal not in totdas_palavras_irrelevantes:
            totdas_palavras_irrelevantes.append(pal)
            


In [214]:
serie_irrelevantes = pd.Series(totdas_palavras_irrelevantes)
tabela_irrelevantes_relativa = serie_irrelevantes.value_counts(True)
tabela_irrelevantes_relativa

e            0.024422
que          0.021828
o            0.021180
de           0.020099
nubank       0.019667
               ...   
padrinho     0.000216
chamei       0.000216
senao        0.000216
reduzir      0.000216
gratuito.    0.000216
Length: 2188, dtype: float64

In [215]:
serie_relevantes = pd.Series(totdas_palavras_relevantes)
tabela_relevantes_relativa = serie_relevantes.value_counts(True)
tabela_relevantes_relativa

e                          0.014718
o                          0.011775
de                         0.010934
meu                        0.010934
que                        0.008410
                             ...   
suficiente.                0.000421
todas                      0.000421
https://t.co/bh9nnb2c7f    0.000421
platinum,                  0.000421
gratuito.                  0.000421
Length: 1923, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [216]:
i = 0
teste["Classificação"] = pd.Series()

for linha in teste["palavras"]:
    p_1 = 1
    p_0 = 1
    p_class = ''
    for palavra in linha:
        if palavra in num_rel:
            #Laplace smoothing
            p_1 *= (num_rel[palavra] + 1)/(len_rel + total_palavras)
            
        else:
            p_1 *= 1/(len_rel + total_palavras)
            
            
        if palavra in num_irrel:
            #Laplace smoothing
            
            p_0 *= (num_irrel[palavra] + 1)/(len_irrel + total_palavras)
        else:
            
            p_0 *= 1/(len_irrel + total_palavras)
            
    if p_1 > p_0:
        p_class = '1'
        teste["Classificação"][i] = p_class
        

    elif p_u1 < p_zero:
        p_class = '0'
        teste["Classificação"][i] = p_class
        
    i+=1


NameError: name 'teste' is not defined

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[StackOverflow](https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python)